In [3]:
####階層ベイズプロビットモデル####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.stats as ss
from numpy.random import *
from scipy import optimize
from scipy.stats import norm

In [4]:
####データの発生####
#np.random.seed(8742)   #シードを設定

##データの設定
hh = 2000   #ユーザー数
k = 7   #パラメータ数
n = poisson(numpy.random.gamma(40, 1/1.5, hh), hh)   #1人あたりの観測数
N = np.sum(n)

In [5]:
##IDの設定
u_list = [i for i in range(hh)]
t_list = [i for i in range(hh)]
u_id = np.zeros(N, dtype='int')
t_id = np.zeros(N, dtype='int')
start = 0

for i in range(hh):
    #リストに格納
    u_list[i] = np.repeat(i, n[i])
    t_list[i] = np.array(range(n[i]))+1
    
    #リストをベクトル変換
    u_id[start:start+n[i]] = u_list[i]
    t_id[start:start+n[i]] = t_list[i]
    start += n[i] 

In [6]:
##説明変数の発生
intercept = np.transpose(np.matlib.repmat(1, 1, N))
Z1 = np.log(numpy.random.gamma(5, 1, N)) * numpy.random.binomial(1, 0.25, N)   #降水量の対数
Z2 = numpy.random.uniform(0, 1, N)   #チラシ商品の平均値引率
Z3 = np.log(numpy.random.poisson(numpy.random.gamma(25, 1/0.5, N)))   #チラシ掲載商品数の対数
Z3 = Z3 - np.min(Z3)
Z4 = numpy.random.binomial(1, 0.5, N)   #他店チラシ状況
Z = np.concatenate((intercept, Z1[:, np.newaxis], Z2[:, np.newaxis], Z3[:, np.newaxis], Z4[:, np.newaxis]), axis=1)   #データの結合

In [7]:
##パラメータの設定
beta0 = numpy.random.normal(-0.4, 0.4, hh)
beta1 = numpy.random.normal(-0.5, 0.3, hh)
beta2 = numpy.random.normal(0.5, 0.25, hh)
beta3 = numpy.random.normal(0.3, 0.25, hh)
beta4 = numpy.random.normal(-0.6, 0.2, hh)
beta = betat = np.concatenate((beta0[:, np.newaxis], beta1[:, np.newaxis], beta2[:, np.newaxis], beta3[:, np.newaxis],
                       beta4[:, np.newaxis]), axis=1)   #回帰係数の結合

In [8]:
##応答変数の発生
mu = np.sum(Z * beta[u_id, :], axis=1)   #潜在効用の平均構造
U = numpy.random.normal(mu, 1, N)   #潜在効用を生成
y = (U >= 0)*1 + (U < 0)*0   #購買有無に変換
np.mean(y)

0.41935605205747384

In [9]:
####マルコフ連鎖モンテカルロ法で階層ベイズプロビットモデルを推定####
##切断正規分布の乱数を発生させる関数
def rtnorm(mu, sigma, a, b, n):
    FA = norm.cdf(a, mu, sigma)
    FB = norm.cdf(b, mu, sigma)
    return norm.ppf(uniform(0, 1, n)*(FB-FA)+FA, mu, sigma)

In [10]:
##アルゴリズムの設定
R = 5000
keep = 4
burnin = int(1000/keep)
disp = 8
iter = 0
k = Z.shape[1]

##事前分布の設定
#プロビットモデルの事前分布
sigma = 1
beta0 = 0
tau0 = 1/100

#階層モデルの事前分布
Bbar = np.repeat(0, k)
A = 0.01 * np.identity(k)
nu = k*2
V = nu * np.identity(k)

In [11]:
##初期値の設定
#回帰係数の初期値
beta_mu = np.dot(np.linalg.inv(np.dot(Z.T, Z)), np.dot(Z.T, y))
beta = np.random.multivariate_normal(beta_mu, 0.1 * np.identity(k), hh)

#階層モデルの初期値
alpha = beta_mu
Cov = 0.1 * np.identity(k)
Cov_inv = np.linalg.inv(Cov)

In [18]:
##パラメータ格納用配列
BETA = np.zeros((hh, k, int(R/keep)))
ALPHA = np.zeros((int(R/keep), k))
COV = np.zeros((k, k, int(R/keep)))

##インデックスを作成
index = np.array(range(N))
index_u = [i for i in range(hh)]

for i in range(hh):
    index_u[i] = index[u_id==i]

##切断領域を定義
a = (1-y)*(-100) + y*0
b = y*100 + (1-y)*0

##定数を設定
XX = [i for i in range(hh)]
XX_inv = [i for i in range(hh)]
for i in range(hh):
    index = index_u[i]
    XX[i] = np.dot(Z[index, :].T, Z[index, :])

In [21]:
####ギブスサンプリングでパラメータをサンプリング####
##切断正規分布から潜在効用を生成
mu = Z * beta[u_id, :]   #潜在効用の平均構造
U = rtnorm(mu, sigma, a, b, N)   #潜在効用を生成

##個人別にbetaをサンプリング



array([[-0.54088245, -0.        ,  0.04338085,  0.68632134, -0.45731496],
       [-0.54088245, -0.        ,  0.0130131 ,  1.14935258, -0.        ],
       [-0.54088245, -0.        ,  0.04688702,  1.29858047, -0.45731496],
       ..., 
       [-1.52897806, -1.43628732, -0.02592791,  0.54018705,  0.        ],
       [-1.52897806, -0.        , -0.10648447,  0.57568837,  0.        ],
       [-1.52897806, -0.        , -0.00333871,  0.51504548,  0.        ]])

In [17]:
np.mean(BETA, axis=2)

array([[-0.45917573, -0.13722834,  0.60844775,  0.38956425, -0.51169244],
       [-0.73758392, -1.09115429,  0.8939035 ,  0.40263903, -1.13042926],
       [-0.42083973,  0.22551533, -0.29366546, -0.32429266,  0.0123086 ],
       ..., 
       [-0.14054832, -0.22709649,  0.77222094,  0.56466766, -0.78025445],
       [-0.73192676, -0.89120465,  0.77204001,  0.90197378, -0.83019372],
       [-0.68311053, -0.85419368,  0.3979096 ,  0.3155925 , -0.5519091 ]])

In [356]:
np.random.multivariate_normal(alpha_mu, Cov/(hh + tau0), 1)

array([[ 0.25634946, -0.22395765,  0.16446298, -0.00753195, -0.29143252]])

In [282]:
i = 3
print(y[i])
print(norm.cdf(mu[i], 0, sigma))
print(scipy.stats.binom.pmf(y[i], 1, norm.cdf(mu[i], 0, sigma)))

0
0.315393440713
0.684606559287
